<img src="https://www.th-koeln.de/img/logo.svg" style="float: right;" width="200">

# 12th exercise: <font color="#C70039">Using recurrent neural networks</font>
* Course: DIS21a.1
* Lecturer: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Author of notebook modifications and adaptations: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Date:   09.09.2024

<img src="https://www.knime.com/sites/default/files/fig_1_3.png" style="float: center;" width="700">

---------------------------------
**GENERAL NOTE 1**: 
Please make sure you are reading the entire notebook, since it contains a lot of information about your tasks (e.g. regarding the set of certain paramaters or specific computational tricks, etc.), and the written mark downs as well as comments contain a lot of information on how things work together as a whole. 

**GENERAL NOTE 2**: 
* Please, when commenting source code, just use English language only. 
* When describing an observation (for instance, after you have run through your test plan) you may use German language.
This applies to all exercises in DIS 21a.1.  

---------------------

### <font color="ce33ff">DESCRIPTION</font>:

This notebook allows for learning how to setup and use a Recurrent Neural Network (RNN). So let's go for it.

-------------------------------------------------------------------------------------------------------------

### <font color="FFC300">TASKS</font>:
Within this notebook, the tasks that you need to work on are always listed as bullet points below. 
If a task is more challenging and consists of several steps, this is indicated as well. 
Make sure you have worked down the task list and commented your doings. 
This should be done using markdown.<br> 
<font color=red>Make sure you don't forget to specify your name and your matriculation number in the notebook before submitting it.</font>

**YOUR TASKS in this exercise are as follows**:
1. import the notebook to Google Colab.
2. make sure you specified you name and your matriculation number in the header below my name and date. 
    * set the date too and remove mine.
3. read the entire notebook carefully. 
    * add comments whereever you feel it necessary for better understanding.
    * run the notebook and try to follow and understand all steps and examples.
4. at the end of PART II, interprete the two charts (training and validation accuracy) and (training and validation loss) and describe your findings.
5. the exercise in this notebook contains in PART III one single implementational task. 
    * implement a Gated Recurrent Unit (GRU) by yourself using Keras' built-in functions.
        * use the same training and validation data set
        * use a history object
        * plot the results of the training and validation accuracy and loss, respectively.
    * let yourself be guided from the other examples in this notebook. 
-----------------------------------------------------------------------------------

## START OF THE NOTEBOOK CODE
---------------------------------------------------

## <font color="#C70039">PART I</font>
### Getting started with RNNs
#### A first recurrent layer in Keras

In [ ]:
import tensorflow
tensorflow.keras.__version__

The process in exercise 11 that was naively implemented in numpy corresponds to an actual Keras layer: the `SimpleRNN` layer.
`SimpleRNN` processes batches of sequences, like all the other Keras layers do, and not just a single sequence like 
in the numpy implementation example of exercise 11. This means that it takes inputs of shape `(batch_size, timesteps, input_features)`, rather than `(timesteps, input_features)`.

As all recurrent layers in Keras, `SimpleRNN` can be run in two different modes. It can either 

* return either the full sequences of successive outputs for each timestep (a 3D tensor of shape `(batch_size, timesteps, output_features)`)

or 

* return only the last output for each input sequence (a 2D tensor of shape `(batch_size, output_features)`). These two modes are controlled by the `return_sequences` constructor argument. 

Let us take a look at an example.

In [ ]:
from tensorflow.keras.layers import SimpleRNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32))
model.summary()

In [ ]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.summary()

It is sometimes useful to stack several recurrent layers one after the other in order to increase the representational power of a network. 
In such a setup, you have to get all intermediate layers to return full sequences:

In [ ]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))  # This last layer only returns the last outputs.
model.summary()

Now, try to use such a model on the IMDB movie review classification problem. 

Step1: preprocess the data

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

max_features = 10000  # number of words to consider as features
maxlen = 500  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(len(input_train), 'train sequences')
print(len(input_test), 'test sequences')

print('Pad sequences (samples x time)')
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape)

Let us train a simple recurrent network using an `Embedding` layer and a `SimpleRNN` layer:

In [ ]:
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['acc'])

history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Let us display the training and validation loss and accuracy:

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Remember that in the first IMDB exercise, the first naive approach reached up to 88% test accuracy (baseline). Unfortunately, this simple RNN does not perform very well at all compared to that baseline since it only reached to 85% validation accuracy. So why is this?

On the one hand, part of the problem is that the inputs only consider the first 500 words rather the full sequences.  
Hence the RNN has got access to less information than the earlier baseline model (please compare). 

On the other hand the problem is simply that `SimpleRNN` is not very good at processing long sequences, like text. 
Other types of recurrent layers perform much better such as LSTM and GRU which you have learned in the lecture course. 

## <font color="#C70039">PART II</font>

### LSTM example implementation in Keras

Now, for the above mentioned reasons it makes sense to set up a model using a LSTM layer and train it on the very same IMDB data. 

<font color="#C70039">NOTE:</font> The network is quite similar to the one with SimpleRNN from Part I above. 

Only specify the output dimensionality of the LSTM layer, and leave every other argument to the Keras defaults. Keras has got good defaults and things will almost always just work without you having to spend time tuning parameters by hand.

In [ ]:
from tensorflow.keras.layers import LSTM

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

<font color="ce33ff">Your TASK</font>

Interprete the two charts and describe them here!!!

----------------
## <font color="#C70039">PART III</font>

### GRU example implementation in Keras

In [ ]:
# add your code here